In [19]:
# 20220209_自定义
size_调单人员 = 514
pagesize = 50
page_i = int(size_调单人员 / pagesize)+1

import os
output_path = 'C:\Users\zqy\Desktop\长城2号\\'
if not os.path.exists(output_path):
    os.mkdir(output_path)
os.chdir(output_path )

import requests
import json
url = 'http://172.30.6.4/ruleResultTest/queryComparedResult'
headers = {'Host': '172.30.6.4',
'Connection': 'keep-alive',
'Content-Length': '75',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
'Content-Type': 'application/json;charset=UTF-8',
'Accept': '*/*',
'Origin': 'http://172.30.6.4',
'Referer': 'http://172.30.6.4/modelTool',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'zh-CN,zh;q=0.9',
'Cookie': 'SHIROJSESSIONID=0ad0a234-2e28-4481-94b7-0ffa335fe6c5'                                        # Cookie
    }
for i in range(page_i):
    body = {"taskId":"a5298a80_d772_4621_83d5_7c9fe43e3629","pageNum":i+1,"pageSize":pagesize}         # taskId
    response = requests.post(url, data = json.dumps(body), headers = headers)
#     print(response.text)
    print(str(i+1)+'Done')

    f2 = open(str(i+1)+'父任务.json', 'w', encoding="utf-8")
    f2.write(response.text)
    f2.close()
    
bankcard_size= 0


import requests
import json
url = 'http://172.30.6.4/ruleResultTest/queryRuleByHitModelCard'
for i in range(page_i):
    with open(str(i+1)+'父任务.json','r', encoding='utf-8')as fp:
        rows = json.load(fp)
    for j in range(len(rows.get('result').get('result'))):
        bankcard_size = bankcard_size + len(rows.get('result').get('result')[j].get('bankList'))       
        for k in range(len(rows.get('result').get('result')[j].get('bankList'))):
            body = {"taskId":rows.get('result').get('result')[j].get('bankList')[k].get('taskId'),
                    "runId":rows.get('result').get('result')[j].get('bankList')[k].get('runId'),
                    "cardNo":rows.get('result').get('result')[j].get('bankList')[k].get('bankcard')
                   }
            headers = {'Host': '172.30.6.4',
'Connection': 'keep-alive',
'Content-Length': '127',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
'Content-Type': 'application/json;charset=UTF-8',
'Accept': '*/*',
'Origin': 'http://172.30.6.4',
'Referer': 'http://172.30.6.4/modelTool',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'zh-CN,zh;q=0.9',
'Cookie': 'SHIROJSESSIONID=0ad0a234-2e28-4481-94b7-0ffa335fe6c5'
            }
            response = requests.post(url, data = json.dumps(body), headers = headers)
#             print(response.text)
            f2 = open(str(rows.get('result').get('result')[j].get('bankList')[k].get('bankcard'))+'.json', 'w', encoding="utf-8")
            f2.write(response.text)
            f2.close()
            print(bankcard_size )
        print(j,end = '-')
    print(i,end = '-')
    
import pandas as pd
df_命中汇总 = pd.DataFrame()
bankcard_size= 0
if not os.path.exists(output_path + 'ruleResult'):
    os.mkdir(output_path + 'ruleResult')
os.chdir(output_path)


url = 'http://172.30.6.4/ruleResultTest/test'
for i in range(page_i):
    with open(str(i+1)+'父任务.json','r', encoding='utf-8')as fp:
        rows = json.load(fp)
    for j in range(len(rows.get('result').get('result'))):
        bankcard_size = bankcard_size + len(rows.get('result').get('result')[j].get('bankList'))   
        for k in range(len(rows.get('result').get('result')[j].get('bankList'))):
            with open(str(rows.get('result').get('result')[j].get('bankList')[k].get('bankcard'))+'.json','r', encoding='utf-8') as fpf:
                json_dict = json.load(fpf)
            for p in range(len(json_dict.get('result').get('result') )):

            
                params = {  
'commonId': '',
'modelId': json_dict.get('result').get('result')[p].get('modelId'),
'cardNo': json_dict.get('result').get('result')[p].get('bankcard'),
'taskId': json_dict.get('result').get('result')[p].get('taskId'),
'runId': json_dict.get('result').get('result')[p].get('runId'),
                }   
                headers = {'Host': '172.30.6.4',
'Connection': 'keep-alive',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
'Accept': '*/*',
'Referer': 'http://172.30.6.4/ruleResult?taskId=216ea370_538d_4707_a0eb_c659e2e8591a&runId=8c8f2c76-5db2-4c81-9a46-319398a25a41&modelId=366eb89b-39bd-4aff-94d9-6cfdc9ca0411&cardNo=32001595536052505827',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'zh-CN,zh;q=0.9',
'Cookie': 'SHIROJSESSIONID=b89f428e-96fa-4abc-9340-103e1523e44c'
                }
                response = requests.get(url ,params = params, headers = headers)
#                 print(response.text)
                text = json.loads(response.text)
    
                ###
                if text.get('result').get('suspiciousDescription')==None:
                    continue
                rule_size= 0
                df = pd.DataFrame(columns = ['卡号','ruleName','hit','finaRuleResult','模型'])
                for x in range(len( text.get('result').get('suspiciousDescription') )):
                    for y in range(len( text.get('result').get('suspiciousDescription')[x].get('ruleList') )):
                        rule_size_start = rule_size
#                         rule_size = rule_size + len(text.get('result').get('suspiciousDescription')[x].get('ruleList')) 
                        n = rule_size_start + y
                        df.loc[n] = { 
                                '卡号':str(json_dict.get('result').get('result')[p].get('bankcard')),
                                'ruleName':text.get('result').get('suspiciousDescription')[x].get('ruleList')[y].get('ruleName'),
                                'hit':text.get('result').get('suspiciousDescription')[x].get('ruleList')[y].get('hit'),
                                'finaRuleResult':text.get('result').get('suspiciousDescription')[x].get('ruleList')[y].get('finalRuleResult'),
                                '模型':str(json_dict.get('result').get('result')[p].get('modelName'))
                                     }
#                         print('第'+str(y),end = '-')
                    rule_size = rule_size + len(text.get('result').get('suspiciousDescription')[x].get('ruleList')) 
#                     print('组'+str(x),end = ' / ')
                df_命中汇总 = df_命中汇总.append(df)
                ###
    
                f2 = open('ruleResult\\'+ str(json_dict.get('result').get('result')[p].get('bankcard'))+str(json_dict.get('result').get('result')[p].get('modelName'))+'.json', 'w', encoding="utf-8")
                f2.write(response.text)
                f2.close()
                print('模型'+str(p),end = ' ')
#             print('银行卡'+str(bankcard_size) )
        print(j)                                            # 调单人数进度,1到50
#     print(i)                                              # 第i个50人里程碑

###
df_hit = df_命中汇总[df_命中汇总['hit']=='1']
df_hit_to_excel('C:\\Users\\Administrator\\Desktop\\0425命中详情-20211019任务.xlsx',index = None)
print('Done!')
###

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 0
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 43
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 35
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 36
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 31
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 32
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 4
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 5
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 

模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 2
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 3
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 4
模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型8 模型9 模型10 模型11 模型12 模型13 模型14 模型15 模型16 模型17 模型18 模型19 模型20 模型21 模型22 模型23 模型24 模型25 模型26 模型27 模型28 模型29 模型30 模型31 模型32 模型33 模型34 模型35 模型36 模型37 模型38 模型39 模型40 模型41 模型42 模型43 模型44 模型45 模型46 模型47 模型48 模型49 模型0 模型1 模型2 模型3 模型4 模型5 模型6 模型7 模型

JSONDecodeError: Extra data: line 1 column 5520 (char 5519)

In [11]:
df_命中汇总

,卡号,ruleName,hit,finaRuleResult,模型
0,32001642336052506511,备注异常,1,备注异常:备注异常，备注包含2036803356、．、．、20201223_01_26643...,通用模型
1,32001642336052506511,频繁发生境外交易,1,频繁发生境外交易:频繁发生境外交易，122.193.248.69(南通)、183.210.1...,通用模型
2,32001642336052506511,公司名称含敏感行业字段,1,公司名称含敏感行业字段:该公司名称包含材料关键字,通用模型
3,32001642336052506511,公司名称含敏感行业字段,1,公司名称含敏感行业字段:该公司名称包含材料关键字,通用模型
4,32001642336052506511,交易对手有同行业公司且交易额较大,0,,通用模型
5,32001642336052506511,交易对手有同行业公司且交易额较大,0,,通用模型
6,32001642336052506511,资金交易类型多样用途明晰,1,资金交易类型多样用途明晰:资金交易类型多样用途明晰，备注包含货款、周攀龙货款、朗盛新材货款、...,通用模型
7,32001642336052506511,资金交易类型多样用途明晰,1,资金交易类型多样用途明晰:资金交易类型多样用途明晰，备注包含货款、周攀龙货款、朗盛新材货款、...,通用模型
8,32001642336052506511,资金交易类型多样用途明晰,1,资金交易类型多样用途明晰:资金交易类型多样用途明晰，备注包含货款、周攀龙货款、朗盛新材货款、...,通用模型
9,32001642336052506511,与企业名称经工商注册核准有较高要求的企业大额交易,1,与企业名称经工商注册核准有较高要求的企业大额交易:交易对手包含中国化工财务有限公司、中国化工...,通用模型


In [40]:
df_命中汇总

,卡号,ruleName,hit,finaRuleResult,模型
0,32001642336052506511,备注异常,1,备注异常:备注异常，备注包含2036803356、．、．、20201223_01_26643...,通用模型
1,32001642336052506511,频繁发生境外交易,1,频繁发生境外交易:频繁发生境外交易，122.193.248.69(南通)、183.210.1...,通用模型
2,32001642336052506511,公司名称含敏感行业字段,1,公司名称含敏感行业字段:该公司名称包含材料关键字,通用模型
5,32001642336052506511,交易对手有同行业公司且交易额较大,0,,通用模型
6,32001642336052506511,资金交易类型多样用途明晰,1,资金交易类型多样用途明晰:资金交易类型多样用途明晰，备注包含货款、周攀龙货款、朗盛新材货款、...,通用模型
10,32001642336052506511,与企业名称经工商注册核准有较高要求的企业大额交易,1,与企业名称经工商注册核准有较高要求的企业大额交易:交易对手包含中国化工财务有限公司、中国化工...,通用模型
14,32001642336052506511,入向交易金额来源集中在少数疑似关系人账户中,0,,通用模型
15,32001642336052506511,日间交易,1,日间交易:日间交易，8点-18点，日间交易笔数20715笔，日间交易占比92.89%，日间交...,通用模型
16,32001642336052506511,中转账户,1,中转账户:账户为中转账户，存在可疑,通用模型
17,32001642336052506511,交易对手有同行业公司且交易额较大,0,,通用模型


In [37]:
df_命中汇总.to_csv('C:\\Users\\Administrator\\Desktop\\0425命中详情-20211019任务-192w.csv')

In [25]:
response.text

'{"code":200,"message":"success","result":{"modelName":"非法经营票据模型","commonName":null,"modelId":"19d55179-30f2-46dd-b1dc-cc16f527c9ab","suspiciousDescription":[{"groupName":"非企业类地方机构","finalGroupResult":"","groupId":"00515a85-9932-462a-9879-6314cc813d1c","ruleList":[{"finalRuleResult":"","hit":"0","ruleName":"非企业类地方机构","ruleId":"gz_1184","etResultTemplate":""}]},{"groupName":"本行业正常经营（物流行业对公）","finalGroupResult":"","groupId":"008a8d2e-b278-4323-bd69-8b5032d154bf","ruleList":[{"finalRuleResult":"","hit":"0","ruleName":"交易对手有同行业公司且交易额较大","ruleId":"gz_1199","etResultTemplate":""},{"finalRuleResult":"","hit":"0","ruleName":"公司名称含敏感行业字段","ruleId":"gz_1162","etResultTemplate":""}]},{"groupName":"真实经营","finalGroupResult":"","groupId":"0c37c812-6aad-4f13-91e2-5f90ed25f7b0","ruleList":[{"finalRuleResult":"","hit":"0","ruleName":"正常纳税","ruleId":"gz_1196","etResultTemplate":""},{"finalRuleResult":"","hit":"0","ruleName":"与企业名称经工商注册核准有较高要求的企业大额交易","ruleId":"gz_1194","etResultTemplate":""},{"finalRule